# Linear Camera Model

This notebook is a study about the linear camera model using *CoppeliaSim* as a simulator to verify results in a virtual environment. 

The goal is to verify if the data generated between math and simulation is coherent. For this:
1. Instanciate a Vision Sensor in both *CoppeliaSim* and in a Mathmatical Model that have matched parameters;
2. Create a 3D scene with detectable points in the space in both environments;
3. Realize perspective projection calculation in the Mathmatical Model;
4. Detect where the points lie in the *CoppeliaSim*'s Vision Sensor image;
5. Check if results match. 

If the results are matching for any position, then the Mathmatical Model fit with *CoppeliaSim*'s image render. 

---

In [2]:
# Importing modules...
import numpy as np
import cv2

import sys
sys.path.append('../..') # Go back to base directory

from modules.view.viewer2d import Viewer2D
from modules.view.viewer3d import Viewer3D

from modules.math.transformations import to_homo
from modules.math.utils import cube

from coppeliasim_zmqremoteapi_client import RemoteAPIClient

# Init client 
client = RemoteAPIClient()    # Client object 
sim = client.getObject('sim') # Simulation object


# Building the Intrinsic Matrix

The **Intrinsic Matrix** is responsible for encoding the camera's internal parameters and how it projects points in the 3D scene to the 2D image plane.

The parameters encoded int he matrix are:
- $x$ and $y$ focal distances in pixels;
- Image plane's principle point.

Also called **Camera Matrix**, it is defined as:

$$K = \begin{bmatrix}
        f_x & 0   & o_x \\
        0   & f_y & o_y \\
        0   & 0   & 1
      \end{bmatrix}$$

The focal distances in pixels are given by:

$$f_x = \frac{R_x}{2\cdot\tan(\frac{\alpha}{2})}$$
$$f_y = \frac{R_y}{2\cdot\tan(\frac{\beta}{2})}$$

And the principle point coordinateas are:
$$o_x = \frac{R_x}{2}$$
$$o_y = \frac{R_y}{2}$$

---

In [3]:
# Declare intrinsic parameters
fov_degrees = 60.0
resolution = (480, 480)

# Generate intrinsic parameters matrix
fov_radians = np.radians(fov_degrees)

# Focal distances in pixels
f_x = resolution[0]/(2*np.tan(fov_radians/2))
f_y = resolution[1]/(2*np.tan(fov_radians/2))

# Principle point
o_x, o_y = resolution[0]/2, resolution[1]/2

intrinsic_matrix = np.array([[f_x,   0, o_x],
                             [  0, f_y, o_y],
                             [  0,   0,   1]])

# Building the Extrinsic Matrix

The **Extrinsic Matrix** encodes the Rotation $R$ and Translation $t$ of the camera frame in a homogeneous transformation:

$$\begin{bmatrix}
    R & t \\
    0 & 1 
\end{bmatrix}_{4\times4}$$

For convenience, the Extrinsic Matrix can be treated as a ${3\times4}$ matrix:

$$\begin{bmatrix}
    R \ | \ t
\end{bmatrix}_{3\times4}$$

The Extrinsic Matrix will be extracted from CoppeliaSim's `sim.getObjectMatrix` API function.

In this model, the $z$-axis must be pointed forward in the optical axis. Looking from behind the camera, the $x$-axis msut be pointing right, and the $y$-axis must be pointing down, just like the image plane. 

---

In [4]:
# Get the vision sensor handle
vision_sensor_handle = sim.getObject('/VisionSensor') 

# Get Coppelia's object matrix
object_matrix = np.array(sim.getObjectMatrix(vision_sensor_handle)).reshape((3,4))

R, t = object_matrix[:, :-1], object_matrix[:, [-1]]

# X and Y axis of Coppelia's Vision Sensor are inverted
R[:,0] *= -1
R[:,1] *= -1

# Generate extrinsic parameters matrix
extrinsic_matrix = np.vstack((np.hstack((R, t)), np.array([0, 0, 0, 1])))

# Setting up and Plotting the 3D Scene

For setting up the scene, there will be a set of 3D points scattered across the Mathmatical Model and *CoppeliaSim*'s environment.

---

In [5]:
# 3D points to be projected in homogeneous coordinates
points_to_project = to_homo(cube()) # Vertices of a cube

# Change cube position
points_to_project[0] -= 0.25
points_to_project[1] -= 0.25
points_to_project[2] += 1

# Create the Scene Viewer
scene = Viewer3D(title='Camera Pose Scene', 
                 size=5)

# Add elements to the scene
scene.add_frame(np.eye(4), 'World Frame', axis_size=0.2)
scene.add_frame(extrinsic_matrix, 'Camera Frame', axis_size=0.2) 
scene.add_points(points_to_project, 'World Points', 'black')

# Plot scene
scene.figure.show(renderer='notebook_connected')

# Perspective Projection

The projection of a 3D Point in the World Frame $P_w$ to a pixel coordinate in the image plane $p_c$ is done by:

$$p_c = \begin{bmatrix}
           K \ | \ 0
        \end{bmatrix}_{3\times4} \cdot \begin{bmatrix}
                                          R & t \\
                                          0 & 1 
                                       \end{bmatrix}_{4\times4} \cdot P_w
$$

or

$$p_c = K \cdot \begin{bmatrix}
                    R \ | \ t
                \end{bmatrix}_{3\times4} \cdot P_w
$$

Both processes will work. Which one is a matter of convenience.

After $p_c$ is obtained, the points will be:
1. Normalized, because they are in homogeneous coordinates;
2. Discarded the last row for 2D points;
3. Casted as intergers, as there are an interger number of pixels.

---

In [6]:
# Generate perspective projection matrix
# Convert intrinsic matrix to 3x4 size and invert the extrinsic parameters
projection_matrix = np.hstack((intrinsic_matrix, np.zeros((3,1)))) @ np.linalg.inv(extrinsic_matrix) 

# Project the 3D point with the new pose
projected_points = projection_matrix @ points_to_project  # Project points to plane
projected_points /= projected_points[-1]                  # Normalize homogeneous coordinates
projected_points = projected_points[:-1, :]               # Discard the last row
projected_points = projected_points.astype(int)           # Cast as interger

# Plotting the 2D Projected Scene

The 3D points will be plotted as 2D projected points in image space. 

---

In [7]:
# Create the image viewer
projection = Viewer2D(title='Projected Points', 
                 resolution=resolution, 
                 graphical=True)

# Adding the projected points to the scene
projection.add_points(projected_points, 'Image Points', 'black')

# Plot projected points
projection.figure.show(renderer='notebook_connected')

# Print Results (Compare with simulation!)
print('Projected Points in Pixel Coordinates:')
print(projected_points)

Projected Points in Pixel Coordinates:
[[247 295 224 280 168 205 126 166]
 [208 133 260 175 162  78 204 107]]


# Getting CoppeliaSim's Image Feed

To detect the point cloud in *CoppeliaSim*, the following process will be followed:

1. Retrieve VisionSensor image with `sim.getVisionSensorCharImage()`;
2. Flip the image;
3. Convert image to gray scale;
4. Threshold the image to evidentiate blobs;
5. Find contours of each blob;
6. Compute centroids of each blob.

---

In [8]:
# When simulation is not running, ZMQ message handling could be a bit
# slow, since the idle loop runs at 8 Hz by default.
# Setting the idle loop to run at full speed for this program
defaultIdleFps = sim.getInt32Param(sim.intparam_idle_fps)   
sim.setInt32Param(sim.intparam_idle_fps, 0)

# Simulation begins here
sim.startSimulation()

point_cloud_handle = sim.createPointCloud(1, 1, 8, 3)
markers_color = list([255,255,255]) 

# Insert points into point cloud
points = np.concatenate(points_to_project[:-1, :].T)
point_count = sim.insertPointsIntoPointCloud(point_cloud_handle, 0, points, markers_color)

detected_centroids = None
image = []

# Other blob detection method
while (t := sim.getSimulationTime()) < 1:
    image, res_x, res_y = sim.getVisionSensorCharImage(vision_sensor_handle)
    image = np.frombuffer(image, dtype=np.uint8).reshape(res_y, res_x, 3) # 3 Channel image buffer (RGB)

    # In CoppeliaSim images are left to right (x-axis), and bottom to top (y-axis)
    # (consistent with the axes of vision sensors, pointing Z outwards, Y up)
    # and color format is RGB triplets, whereas OpenCV uses BGR:
    image = cv2.flip(image, 0)

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Convert the grayscale image to binary image
    _,thresh = cv2.threshold(gray_image,230,255,0)

    # Find blob contours
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    if contours: # Check if empty
        for c in contours:
            # Calculate moments of each contour
            M = cv2.moments(c)
            
            # Calculate x,y coordinate of center
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
            else:
                print('Blob not detected!')
                break
            
            centroids = np.array([cX, cY]).reshape(-1,1)

            if detected_centroids is None:
                detected_centroids = centroids
            else:
                detected_centroids = np.hstack((detected_centroids, centroids))
    else:
        print('Did not detect any blobs...')

    break # Just execute the first loop

# Restore the original idle loop frequency:
sim.setInt32Param(sim.intparam_idle_fps, defaultIdleFps)

# Simulation ends here
sim.stopSimulation()

# Plotting the Vision Sensor's Image 

The Vision Sensor's retrieved image will be plotted for comparison. 

---

In [9]:
# Create the image viewer
image = Viewer2D(title='Coppelia\'s Vision Sensor', 
                 resolution=resolution, 
                 image=image,
                 graphical=True)

# Plot image
image.figure.show(renderer='notebook_connected')

# Print Results (Compare with math model!)
print('Detected Points in Pixel Coordinates:')
print(detected_centroids)

Detected Points in Pixel Coordinates:
[[224 247 126 280 168 295 166 205]
 [260 208 204 175 162 133 107  78]]
